In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib.pylab as plt

from image_analysis.read_files import get_paths, change_paths
from image_analysis.fit import plot, roi, regionprops
from image_analysis.preprocess import mask

from tqdm import tqdm

In [3]:
pixel_size = 2

In [4]:
folder_pretreated = r'example_pretreated_images/'

In [5]:
img_paths = get_paths(folder_pretreated, exclude='.lif_', suffix='.npy')
img_paths = change_paths(img_paths, '', 'pixel')
img_paths = change_paths(img_paths, '', 'keys')

names = [os.path.basename(file).split('.npy')[0] for file in img_paths]

In [6]:
df = pd.DataFrame(names, columns=['pretreated_file_name'])
df['hcr'] = [ int(name.split('_')[0].split('hcr')[1]) for name in df['pretreated_file_name'] ]

In [7]:
df

,pretreated_file_name,hcr
0,hcr21_2_5cm_7dpa_noif,21
1,hcr21_2_5cm_6dpa_p_hist,21


In [8]:
for pc in [40, 50, 60, 70, 80]:

    df[f'system_size_L_along_SF_pc{pc}_um']  = pd.NA
    df[f'length_L_SF_pc{pc}_um']  = pd.NA
    
    df[f'x0_pc{pc}_um_shh']  = pd.NA
    df[f'x0_pc{pc}_um_fgf'] = pd.NA

In [9]:
for i, file_name in tqdm( enumerate( df['pretreated_file_name'].values) ):
    
    file = os.path.join( folder_pretreated, file_name) + '.npy'
    
    keys = np.load( file.split('.npy')[0] + '_keys.npy' )
    image = dict(zip(
        keys,
        np.load( file )
    ))

    for pc in [40, 50, 60, 70, 80]:
        if (f'shh{pc}_clip' in keys) and (f'fgf{pc}_clip' in keys):
            
            image_to_rotate = dict(zip(
                ['shh_clip', 'fgf_clip', 'tissue_mask_with_epi'],
                [image[f'shh{pc}_clip'], image[f'fgf{pc}_clip'], image['tissue_mask_with_epi']]
            ))
             
            regionprops_3d = regionprops.regionprops_3d(image_to_rotate, 1, 1, 1, pixel_size)
            image_to_rotate = regionprops_3d.rotate_sf_to_ey()

            S = regionprops.center_mass( image_to_rotate['shh_clip'] )
            F = regionprops.center_mass( image_to_rotate['fgf_clip'] )

            L = np.sum( 
                mask.largest_component( image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]  )
            ) * pixel_size
            L_SF  = np.abs( S[1] - F[1] ) * pixel_size
            
            ymin = np.nonzero( mask.largest_component(image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]) )[0][0]
            ymax = np.nonzero( mask.largest_component(image_to_rotate['tissue_mask_with_epi'][S[0], :, S[2]]) )[0][-1]            
            
            x0_S = min( (S[1] - ymin) * pixel_size, (ymax - S[1]) * pixel_size )
            x0_F = min( (F[1] - ymin) * pixel_size, (ymax - F[1]) * pixel_size )

            df.loc[df['pretreated_file_name'] == file_name, f'system_size_L_along_SF_pc{pc}_um']  = L
            df.loc[df['pretreated_file_name'] == file_name, f'length_L_SF_pc{pc}_um']             = L_SF
            df.loc[df['pretreated_file_name'] == file_name, f'x0_pc{pc}_um_shh']                  = x0_S
            df.loc[df['pretreated_file_name'] == file_name, f'x0_pc{pc}_um_fgf']                  = x0_F
            
df.to_excel("system_size_L_and_gaps_x0/system_size_L_and_gaps_x0.xlsx", index=False)

0it [00:00, ?it/s]/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


rotation around x:
-4.5739212599008 0 0
dictionary, with 3d data
dictionary, with 3d data


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


rotation around y:
0 0 25.85607883545016
dictionary, with 3d data
dictionary, with 3d data
strange rotation
angle beetween ey and sf: 10


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


...

rotation around x:
-5.856013585428968 0 0
dictionary, with 3d data
dictionary, with 3d data
rotation around y:
0 0 76.77895123478159
dictionary, with 3d data
dictionary, with 3d data
angle beetween ey and sf: 1


/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:25: RuntimeWarning: Mean of empty slice
  img_arr_x = np.nanmean(np.nanmean(img_arr, axis=0), axis=0)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:28: RuntimeWarning: Mean of empty slice
  img_arr_y = np.nanmean(np.nanmean(img_arr, axis=0), axis=1)
/home/natalia/Documents/github_image_analysis/image_analysis/image_analysis/fit/regionprops.py:31: RuntimeWarning: Mean of empty slice
  img_arr_z = np.nanmean(np.nanmean(img_arr, axis=1), axis=1)


rotation around x:
5.906141113770502 0 0
dictionary, with 3d data
dictionary, with 3d data
rotation around y:
0 0 70.30276413857383
dictionary, with 3d data
dictionary, with 3d data
angle beetween ey and sf: 1


2it [00:38, 19.40s/it]


OSError: Cannot save file into a non-existent directory: 'system_size_L_and_gaps_x0'